In [92]:
# !pip install yfinance

In [91]:
import yfinance as yf


def getStockPriceDataframe(tickerSymbol, startDate, endDate):
  # fetch data
  data = yf.Ticker(tickerSymbol).history(start=startDate, end=endDate)

  # drop unnecessary cols
  data = data.drop('Dividends', axis=1)
  data = data.drop('Stock Splits', axis=1)

  # reindex
  data['Timestamp'] = data.index.array
  data.index = range(0, len(data))

  # calc derrived attributes
  data['Delta'] = data['Close'] - data['Open']
  data['Volatility'] = 0.0
  data['AvgVolatility'] = 0.0
  data['Momentum'] = 0.0

  for idx in range(1, len(data)):
    data.at[idx, 'Volatility'] = (data.at[idx - 1, 'Close'] - data.at[idx, 'Close']) / data.at[idx - 1, 'Close']
    data.at[idx, 'Momentum'] = 1.0 if (data.at[idx, 'Close'] > data.at[idx - 1, 'Close']) else 0.0
    if idx > 5:
      data.at[idx, 'AvgVolatility'] = data.iloc[idx-5:idx]['Volatility'].mean()
  return data


df = getStockPriceDataframe(tickerSymbol='ADBE', startDate='2020-03-01', endDate='2020-06-30')
df.to_csv('stock_prices.csv')